## 0. Setting

In [ ]:
# Standard libraries
import os
import gc
import glob
import re
from typing import List

# Third-party libraries
import geopandas as gpd
import mapclassify
import networkx as nx
import numpy as np
import pandas as pd
from shapely.geometry import Point
from shapely.ops import nearest_points

# Visualization libraries (Matplotlib)
import matplotlib.image as mimg
import matplotlib.pyplot as plt
from matplotlib import colors as mcolors
from matplotlib import font_manager as fm
from matplotlib import patheffects as path_effects
from matplotlib.collections import LineCollection
from matplotlib.lines import Line2D
from matplotlib.offsetbox import AnnotationBbox, OffsetImage
from matplotlib.patches import FancyArrowPatch, Patch

if os.getcwd().endswith('notebooks'):
    os.chdir('..')


# Set font path with absolute path (adjust filename/path accordingly)
font_regular = os.path.abspath("assets/malgun.ttf")     # Regular
font_bold    = os.path.abspath("assets/malgunbd.ttf")   # Bold

# Register fonts in Matplotlib font manager
fm.fontManager.addfont(font_regular)
fm.fontManager.addfont(font_bold)

# Safely get the family name from the font file and apply globally
fam = fm.FontProperties(fname=font_regular).get_name()  # Usually 'Malgun Gothic'
plt.rcParams.update({
    "font.family": fam,
    "axes.unicode_minus": False,  # Prevent minus sign from breaking
})

ARROW_FILE = "north_arrow.png"

def add_north_arrow(ax, x, y, arrow_file, zoom=0.05):
    """Add a north arrow image at the given axes position."""
    im = mimg.imread(arrow_file)
    ax.add_artist(
        AnnotationBbox(OffsetImage(im, zoom=zoom), (x, y), 
                       xycoords='axes fraction', frameon=False)
    )

def add_scale_bar(ax, length, location=(0.1, 0.02), linewidth=3, color='black'):
    """Add a scale bar with text in kilometers."""
    xlim, ylim = ax.get_xlim(), ax.get_ylim()
    sb_x = xlim[0] + (xlim[1] - xlim[0]) * location[0]
    sb_y = ylim[0] + (ylim[1] - ylim[0]) * location[1]
    ax.plot([sb_x, sb_x + length], [sb_y, sb_y], color=color, linewidth=linewidth)
    ax.text(sb_x + length/2, sb_y, f'{round(length/1000):,} km',
            va='bottom', ha='center', fontsize=10)


## 1. Data Preparation

In [ ]:
SGG_map = gpd.read_file('data/processed/map/SGG_map.gpkg').to_crs(epsg=5179)
SGG_map['centroid'] = SGG_map.geometry.centroid

# Collect all matching CSV files
all_files = glob.glob("data/processed/deal_network/deal_by/network_by_*.csv")

# Dictionary to store DataFrames
nw_dict = {}

for file in all_files:
    # Extract a key from the file name
    # e.g., 'network_by_age (man).csv' → 'age (man)'
    key = os.path.basename(file).replace("network_by_", "").replace(".csv", "")
    
    # Read the CSV file
    nw = pd.read_csv(
        file,
        dtype={'14_시군구코드_buyer': str, '8_시군구코드_seller': str, '1_기준연도': int}
    )
    nw.columns = ['year', 'source', 'target', '거래관계']
    
    # Merge with centroid coordinates (seller and buyer separately)
    nw = pd.merge(
        nw, SGG_map[['SIG_CD', 'centroid']],
        left_on='source', right_on='SIG_CD', how='left'
    )
    nw = pd.merge(
        nw, SGG_map[['SIG_CD', 'centroid']],
        left_on='target', right_on='SIG_CD', how='left',
        suffixes=('_seller', '_buyer')
    )
    nw = nw.drop(columns=['SIG_CD_seller', 'SIG_CD_buyer'])
    
    # Store DataFrame in dictionary
    nw_dict[key] = nw
    

# Define a preferred order of network groups
desired_order = [
    'all',
    'man',
    'innovation',
    'urban_size_소상공인',
    'urban_size_중소기업',
    'urban_size_중견기업',
    'urban_size_대기업',
    'urban_age_1년 미만',
    'urban_age_1~5년 미만',
    'urban_age_5~10년 미만',
    'urban_age_10년 이상'
]

# Create a new dictionary following the desired order
# Only include keys that actually exist in nw_dict
ordered_nw_dict = {key: nw_dict[key] for key in desired_order if key in nw_dict}

# Replace nw_dict with the ordered dictionary
nw_dict = ordered_nw_dict


# ===== Utility function =====
def build_color_table(max_id=600):
    """
    Build a color table for unique IDs (e.g., for labeling node categories).

    Parameters
    ----------
    max_id : int, optional
        Maximum number of IDs to assign colors to (default is 600).

    Returns
    -------
    dict
        A mapping {id: RGBA color} for each unique ID.
    """
    pal = list(plt.get_cmap("Set1").colors) \
        + list(plt.get_cmap("Dark2").colors) \
        + list(plt.get_cmap("Accent").colors)
    return {cid: mcolors.to_rgba(pal[cid % len(pal)], 1.0) for cid in range(max_id)}


COLOR_OF = build_color_table(600)


## 2. Data table

In [ ]:
import os
import re
import gc
import numpy as np
import pandas as pd
import geopandas as gpd

# ========= Paths / Parameters =========
BASE_DEAL_DIR = "data/processed/deal_network/deal_by"
COMM_BASE_DIR = "outputs/gpkg/communities"
if not os.path.exists(COMM_BASE_DIR):
    os.makedirs(COMM_BASE_DIR, exist_ok=True)
TABLE_DIR     = "outputs/tables/communities_DII_RSI"   # <-- Output folder for results (RSI/DII)
if not os.path.exists(TABLE_DIR):
    os.makedirs(TABLE_DIR, exist_ok=True)


COMMUNITY_METHODS = ['Infomap', "Leiden", "Louvain"]
WEIGHTS           = ["거래관계"]

# ===== Utility functions =====
def compute_dii_rsi_community(ed: pd.DataFrame, lab_map: dict, weight_col: str, year: int):
    """Compute RSI and DII (at the community level)"""
    ed_use = ed.dropna(subset=["source","target",weight_col]).copy()
    ed_use[weight_col] = pd.to_numeric(ed_use[weight_col], errors="coerce").fillna(0)

    # --- map to community ---
    ed_use["m_s"] = ed_use["source"].map(lab_map)
    ed_use["m_t"] = ed_use["target"].map(lab_map)
    ed_use = ed_use.dropna(subset=["m_s","m_t"])
    ed_use = ed_use[ed_use["m_s"] != ed_use["m_t"]]

    if ed_use.empty:
        return pd.DataFrame(), pd.DataFrame()

    # --- aggregate flows at community level ---
    flows = ed_use.groupby(["m_s","m_t"])[weight_col].sum().reset_index()

    # RSI (edge-level, community)
    total_flow = flows[weight_col].sum()
    flows["RSI"] = (flows[weight_col] / total_flow * 100) if total_flow > 0 else 0.0
    rsi_df = flows.copy()
    rsi_df["year"] = year

    # DII (node-level, community)
    inflow = flows.groupby("m_t")[weight_col].sum()
    outflow = flows.groupby("m_s")[weight_col].sum()
    io_sum = inflow.add(outflow, fill_value=0)
    mean_io = io_sum.mean() if not io_sum.empty else 1.0
    dii = io_sum / mean_io if mean_io > 0 else io_sum
    dii_df = dii.rename("DII").reset_index().rename(columns={dii.index.name:"community"})
    dii_df["year"] = year

    return rsi_df, dii_df


# ===== Main loop =====
for key, ed in nw_dict.items():
    all_rsi, all_dii = [], []

    for weight in WEIGHTS:
        gpkg = os.path.join(COMM_BASE_DIR, key, f"{weight}_SGG_map_zone_community.gpkg")
        if not os.path.exists(gpkg): 
            continue

        g = gpd.read_file(gpkg)
        if "SIGUNGU_CD" not in g.columns:
            print(f"[SKIP] {key}/{weight}: SIGUNGU_CD not found")
            continue
        g["SIGUNGU_CD"] = g["SIGUNGU_CD"].astype(str)

        # Check available years
        years = sorted({int(c.split("_")[-1]) for c in g.columns 
                        if c.startswith(tuple(f"{m}_canon_" for m in COMMUNITY_METHODS))})
        if not years: 
            print(f"[SKIP] {key}/{weight}: no canonical years")
            continue

        # Check edge validity
        if not all(c in ed.columns for c in ['year','source','target',weight]):
            print(f"[SKIP] {key}: edges missing required columns")
            continue

        ed_use = ed.dropna(subset=["source","target","year"]).copy()
        ed_use["source"] = ed_use["source"].astype(str)
        ed_use["target"] = ed_use["target"].astype(str)
        ed_use[weight] = pd.to_numeric(ed_use[weight], errors="coerce").fillna(0)

        for method in COMMUNITY_METHODS:
            for y in years:
                col_canon = f"{method}_canon_{y}"
                if col_canon not in g.columns: 
                    continue

                # Map SIGUNGU to community
                s_year = g[["SIGUNGU_CD", col_canon]].dropna(subset=[col_canon])
                s_year[col_canon] = s_year[col_canon].astype(int)
                lab_map = s_year.set_index("SIGUNGU_CD")[col_canon].to_dict()

                ede = ed_use[ed_use["year"] == y].copy()
                if ede.empty: 
                    continue

                # Compute RSI/DII at community level
                rsi_df, dii_df = compute_dii_rsi_community(ede, lab_map, weight, year=y)
                if rsi_df.empty and dii_df.empty:
                    continue
                
                # Natural breaks
                if not dii_df.empty:
                    try:
                        dii_classifier = mapclassify.NaturalBreaks(dii_df["DII"], k=3)
                        dii_df["DII_class"] = 3 - dii_classifier.yb  # Classes start from 1
                    except Exception as e:
                        print(f"[WARN] DII classification failed for {key}/{method}/{y}: {e}")
                        dii_df["DII_class"] = np.nan
                
                rsi_df["key"] = key
                rsi_df["method"] = method
                dii_df["key"] = key
                dii_df["method"] = method

                all_rsi.append(rsi_df)
                all_dii.append(dii_df)

    # ---- Save results (per key) ----
    if all_rsi:
        RSI_OUT = os.path.join(TABLE_DIR, f"rsi_{key}.csv")
        pd.concat(all_rsi, ignore_index=True).to_csv(RSI_OUT, index=False, encoding="cp949")
        print(f"[OK] RSI saved to {RSI_OUT}")
    else:
        print(f"[WARN] No RSI results for {key}")

    if all_dii:
        DII_OUT = os.path.join(TABLE_DIR, f"dii_{key}.csv")
        pd.concat(all_dii, ignore_index=True).to_csv(DII_OUT, index=False, encoding="cp949")
        print(f"[OK] DII saved to {DII_OUT}")
    else:
        print(f"[WARN] No DII results for {key}")


# 3. Inter commuity visualization

In [ ]:
# ========= Paths / Parameters =========
BASE_DEAL_DIR = "data/processed/deal_network/deal_by"  # network_by_*.csv
COMM_BASE_DIR = "outputs/gpkg/communities"             # …/{key}/{weight}_SGG_map_zone_community.gpkg
OUT_DIR       = "outputs/figures/communities/inter_flow"

# What to visualize
COMMUNITY_METHODS = ['Infomap', "Leiden", "Louvain"]
WEIGHTS           = ["거래관계"]  # Column name in the original CSV used as edge weight

# Link / Node style
BASE_LW      = 0.5
LW_RANGE     = 4.0
RSI_W_BLEND  = 0.30   # Blend 30% of RSI into edge width
N_SEG        = 24     # Divide curve into N segments for gradient coloring
CURVATURE    = 0.20   # Bézier curve curvature

# Node size parameters
BASE_NODE_SIZE = 200  # Node size (scatter “s” parameter) when DII = 1.0
DII_CLIP_HI    = 4    # Upper clip value for DII

# RSI → alpha transformation
MIN_ALPHA        = 0.8
MAX_ALPHA        = 1
RSI_CLIP_LO      = 0.0
RSI_CLIP_HI_QUANT = 0.99             # Use the 99th percentile as the upper clip
RSI_PCT_LABELS    = [0.70,0.80,0.90,0.99]  # Percentile labels for the legend

# ===== Utility functions =====
def _curve_segments(p1: Point, p2: Point, curvature=CURVATURE, nseg=N_SEG):
    # Generate curved line segments between two points using a quadratic Bézier curve
    mx, my = (p1.x+p2.x)/2, (p1.y+p2.y)/2
    dx, dy = (p2.x-p1.x), (p2.y-p1.y)
    dist = np.hypot(dx, dy)
    if dist == 0: return None
    px, py = -dy/dist, dx/dist
    cx, cy = mx + px*dist*curvature, my + py*dist*curvature
    t = np.linspace(0, 1, nseg+1)
    bx = (1-t)**2*p1.x + 2*(1-t)*t*cx + t**2*p2.x
    by = (1-t)**2*p1.y + 2*(1-t)*t*cy + t**2*p2.y
    segs = np.stack([np.column_stack([bx[:-1], by[:-1]]),
                     np.column_stack([bx[1:],  by[1:]])], axis=1)
    return segs

def _norm01(a):
    # Normalize array values to [0, 1]
    a = np.asarray(a, float)
    if a.size == 0: return a
    vmin, vmax = np.nanmin(a), np.nanmax(a)
    if not np.isfinite(vmin) or not np.isfinite(vmax) or vmax <= vmin:
        return np.full_like(a, 0.5)
    return (a - vmin) / (vmax - vmin)

def _alpha_from_rsi(rsi, clip_lo=RSI_CLIP_LO, clip_hi=None):
    # Convert RSI values into alpha transparency with clipping
    rsi = np.asarray(rsi, float)
    if clip_hi is None or not np.isfinite(clip_hi):
        clip_hi = np.quantile(rsi, RSI_CLIP_HI_QUANT) if np.isfinite(np.nanmax(rsi)) else 0.01
        clip_hi = max(clip_hi, clip_lo * 10)
    r = np.clip(rsi, clip_lo, clip_hi)
    z = (r - clip_lo) / (clip_hi - clip_lo) if clip_hi > clip_lo else np.zeros_like(r)
    z = np.clip(z, 0, 1)
    return MIN_ALPHA + (MAX_ALPHA - MIN_ALPHA) * z


# ===== Main loop =====
os.makedirs(OUT_DIR, exist_ok=True)

for key, ed in nw_dict.items():
    for weight in WEIGHTS:
        gpkg = os.path.join(COMM_BASE_DIR, key, f"{weight}_SGG_map_zone_community.gpkg")
        if not os.path.exists(gpkg): continue

        g = gpd.read_file(gpkg)
        if "SIGUNGU_CD" not in g.columns:
            print(f"[SKIP] {key}/{weight}: SIGUNGU_CD not found")
            continue
        g["SIGUNGU_CD"] = g["SIGUNGU_CD"].astype(str)

        # Extract available years from canonical columns
        years = sorted({int(c.split("_")[-1]) for c in g.columns if c.startswith(tuple(f"{m}_canon_" for m in COMMUNITY_METHODS))})
        if not years:
            print(f"[SKIP] {key}/{weight}: no canonical years found")
            continue
        
        # Validate edge table
        if all(c in ed.columns for c in ['year', 'source', 'target', weight]):
            ed_use = ed.dropna(subset=["source", "target", "year"]).copy()
            ed_use["source"] = ed_use["source"].astype(str)
            ed_use["target"] = ed_use["target"].astype(str)
            ed_use[weight] = pd.to_numeric(ed_use[weight], errors="coerce").fillna(0)
        else:
            print(f"[SKIP] {key}: edges missing required columns")
            continue

        for method in COMMUNITY_METHODS:
            canon_cols = [c for c in g.columns if c.startswith(f"{method}_canon_")]
            if not canon_cols: continue

            # ---- Build master network (integrated across all years) ----
            all_nodes_master = set()
            all_flows_master = {}

            for y_build in years:
                col_canon_build = f"{method}_canon_{y_build}"
                if col_canon_build not in g.columns: continue

                s_year_build = g[["SIGUNGU_CD", col_canon_build]].dropna(subset=[col_canon_build])
                s_year_build[col_canon_build] = s_year_build[col_canon_build].astype(int)
                all_nodes_master.update(s_year_build[col_canon_build].unique())

                lab_map_build = s_year_build.set_index("SIGUNGU_CD")[col_canon_build].to_dict()
                ede_build = ed_use[ed_use["year"] == y_build].copy()
                if not ede_build.empty:
                    ede_build["m_s"] = ede_build["source"].map(lab_map_build)
                    ede_build["m_t"] = ede_build["target"].map(lab_map_build)
                    ede_build = ede_build.dropna(subset=["m_s", "m_t"])
                    ede_build = ede_build[ede_build["m_s"] != ede_build["m_t"]]
                    flows_build = ede_build.groupby(["m_s", "m_t"])[weight].sum()
                    for (ms, mt), val in flows_build.items():
                        all_flows_master[(int(ms), int(mt))] = all_flows_master.get((int(ms), int(mt)), 0) + val
            
            G_master = nx.DiGraph()
            G_master.add_nodes_from(all_nodes_master)
            for (ms, mt), val in all_flows_master.items():
                G_master.add_edge(ms, mt, weight=val)

            # Global layout
            master_pos = {}
            if G_master.number_of_nodes() > 0:
                print(f"Calculating master layout for [{key}/{weight}/{method}]...")
                master_pos = nx.kamada_kawai_layout(G_master, scale=2.0)
            
            # ---- Fix axis ranges ----
            master_xlim = (-1.2, 1.2)
            master_ylim = (-1.2, 1.2)
            if master_pos:
                x_coords = [p[0] for p in master_pos.values()]
                y_coords = [p[1] for p in master_pos.values()]
                x_min, x_max = min(x_coords), max(x_coords)
                y_min, y_max = min(y_coords), max(y_coords)
                x_margin = (x_max - x_min) * 0.15 if (x_max > x_min) else 0.2
                y_margin = (y_max - y_min) * 0.15 if (y_max > y_min) else 0.2
                master_xlim = (x_min - x_margin, x_max + x_margin)
                master_ylim = (y_min - y_margin, y_max + y_margin)
            
            # ---- Yearly visualization ----
            for y in years:
                outdir = os.path.join(OUT_DIR, key, weight)
                os.makedirs(outdir, exist_ok=True)
                safe = lambda s: re.sub(r"[^0-9A-Za-z가-힣_.\\-]+", "_", s)
                outpath = os.path.join(outdir, f"flows_{safe(method)}_{y}.png")

                if os.path.exists(outpath):
                    print(f"  [SKIP] {key}/{weight}: {method} Y{y} already exists")
                    continue

                col_canon = f"{method}_canon_{y}"
                if col_canon not in g.columns: continue
                
                s_year = g[["SIGUNGU_CD", col_canon]].dropna(subset=[col_canon])
                s_year[col_canon] = s_year[col_canon].astype(int)
                lab_map = s_year.set_index("SIGUNGU_CD")[col_canon].to_dict()
                size_of = s_year.groupby(col_canon).size().to_dict()
                
                ede = ed_use[ed_use["year"] == y].copy()
                if ede.empty: flows = {}
                else:
                    ede["m_s"] = ede["source"].map(lab_map)
                    ede["m_t"] = ede["target"].map(lab_map)
                    ede = ede.dropna(subset=["m_s","m_t"])
                    ede["m_s"] = ede["m_s"].astype(int)
                    ede["m_t"] = ede["m_t"].astype(int)
                    ede = ede[ede["m_s"] != ede["m_t"]]
                    flows = ede.groupby(["m_s","m_t"])[weight].sum().to_dict()

                # RSI / DII 계산
                if flows:
                    v_arr = np.array(list(flows.values()), float)
                    total_flow = float(np.nansum(v_arr))
                    rsi = {k: (val / total_flow if total_flow > 0 else 0.0) for k, val in flows.items()}
                    rsi_vals = np.array(list(rsi.values()), float)
                    clip_hi  = np.quantile(rsi_vals, RSI_CLIP_HI_QUANT) if rsi_vals.size else 0.01
                    alphas   = {k: float(_alpha_from_rsi([rsi[k]], clip_lo=RSI_CLIP_LO, clip_hi=clip_hi)[0]) for k in rsi}
                    vmax = np.nanmax(v_arr) if np.isfinite(np.nanmax(v_arr)) else 1.0
                    logv = np.log10((v_arr / vmax) * 9.0 + 1.0) if vmax > 0 else np.zeros_like(v_arr)
                    w_norm = _norm01(logv)
                    rsi01 = {k: (alphas[k] - MIN_ALPHA) / (MAX_ALPHA - MIN_ALPHA) if (MAX_ALPHA - MIN_ALPHA) > 0 else 0 for k in rsi}
                    widths = {k: BASE_LW + LW_RANGE * ((1.0 - RSI_W_BLEND) * w_norm[i] + RSI_W_BLEND * rsi01[k]) for i, k in enumerate(flows.keys())}
                else:
                    rsi, alphas, widths, rsi_vals = {}, {}, {}, np.array([])

                if flows:
                    io_sum = {}
                    for (ms, mt), val in flows.items():
                        io_sum[ms] = io_sum.get(ms, 0.0) + float(val)
                        io_sum[mt] = io_sum.get(mt, 0.0) + float(val)
                    vals = np.array(list(io_sum.values()), float)
                    mean_io = float(vals.mean()) if vals.size else 1.0
                    dii = {c: (io_sum.get(c, 0.0) / mean_io if mean_io > 0 else 0.0) for c in set(io_sum.keys())}
                else:
                    dii = {}
                
                node_ids  = sorted(list(size_of.keys()))
                node_cols = {cid: COLOR_OF[cid] for cid in node_ids}
                if dii:
                    node_sizes = {cid: BASE_NODE_SIZE * min(DII_CLIP_HI, dii.get(cid, 1.0)) for cid in node_ids}
                else:
                    node_sizes = {cid: BASE_NODE_SIZE for cid in node_ids}
                
                pos_this_year = {nid: master_pos[nid] for nid in node_ids if nid in master_pos}
                node_pts = {cid: Point(p) for cid, p in pos_this_year.items()}
                
                fig, ax = plt.subplots(figsize=(6, 6))
                ax.axis("off")
                ax.set_xlim(master_xlim)
                ax.set_ylim(master_ylim)

                # Draw edges
                for (ms, mt), val in flows.items():
                    if ms not in node_pts or mt not in node_pts: continue
                    p1, p2 = node_pts[ms], node_pts[mt]
                    segs = _curve_segments(p1, p2, curvature=CURVATURE, nseg=N_SEG)
                    if segs is None: continue
                    c0 = mcolors.to_rgba(node_cols.get(ms, (0.2,0.2,0.2,1.0)), 1.0)
                    c1 = mcolors.to_rgba(mcolors.to_rgb(node_cols.get(ms, (0.2,0.2,0.2,1.0))), 0.1)
                    cols = np.array([np.linspace(c0[i], c1[i], N_SEG) for i in range(4)]).T
                    cols[:, 3] *= alphas.get((ms, mt), 0.6)
                    lc = LineCollection(segs, colors=cols,
                                        linewidths=widths.get((ms,mt), BASE_LW),
                                        capstyle="round", zorder=1)
                    ax.add_collection(lc)

                # Draw nodes
                for cid, pt in node_pts.items():
                    if pt is None: continue
                    s = node_sizes.get(cid, BASE_NODE_SIZE)
                    ax.scatter([pt.x],[pt.y], s=s, c=[node_cols[cid]],
                               edgecolors="white", linewidths=0.9,
                               alpha=0.95, zorder=3)
                
                # Node labels
                for cid, pt in node_pts.items():
                    if pt is None: continue
                    label = f"M{cid}"
                    t = ax.text(pt.x, pt.y, label, fontsize=8.5,
                                color="black", ha="center", va="center", zorder=4)
                    t.set_path_effects([path_effects.Stroke(linewidth=2.4,
                                                           foreground="white", alpha=0.8),
                                        path_effects.Normal()])

                ax.set_title(f"[{key}] {weight} · {method} · Y{y}\nCommunity flows", fontsize=13, y=1.02)

                # ---- 범례 ----
                handles_main = []
                # DII legend
                handles_main.append(Line2D([0],[0], color="none", label="Dominance Index (DII)"))
                DII_LEGEND_SHRINK = 0.3
                legend_dii_levels = [0.5, 1.0, 2.0, DII_CLIP_HI]
                legend_dii_levels = [lv for lv in legend_dii_levels if lv <= DII_CLIP_HI]
                legend_dii_labels = sorted(list(set([l for l in [0.5, 1.0, 2.0, DII_CLIP_HI] if l <= DII_CLIP_HI])))
                for lab in legend_dii_labels:
                    ms = BASE_NODE_SIZE * lab 
                    handles_main.append(Line2D([], [], marker="o", linestyle="None", markersize=np.sqrt(ms), markerfacecolor="#66bb6a", markeredgecolor="white", markeredgewidth=0.8, label=f"  ×{lab:g}"))
                    handles_main.append(Line2D([0],[0], color="none", linewidth=0.2, label = None))

                # RSI legend
                handles_main.append(Line2D([0],[0], color="none", label="RSI (percentile)"))
                if flows and len(rsi_vals) > 0:
                    pct_targets = RSI_PCT_LABELS
                    keys_list = list(flows.keys())
                    vals_list = list(rsi.values())
                    for p in pct_targets:
                        rv = np.quantile(rsi_vals, p)
                        idx = np.argmin(np.abs(np.array(vals_list) - rv))
                        k_demo = keys_list[idx]
                        alpha_val = alphas.get(k_demo, 0.9)
                        width_demo = widths.get(k_demo, BASE_LW)
                        handles_main.append(
                            Line2D([0],[0], color=(0,0,0,alpha_val),
                                   lw=width_demo, solid_capstyle="round",
                                   label=f"  {vals_list[idx]*100:.2f}% (P{int(p*100)})")
                        )
                else:
                    handles_main.append(Line2D([0],[0], color="none", label="  (no edges)"))

                # Source/Target
                handles_main.append(Line2D([0],[0], color="none", label="Source / Target"))
                handles_main.append(Patch(facecolor="black", edgecolor="black", label="  Source"))
                handles_main.append(Patch(facecolor="white", edgecolor="black", label="  Target"))

                leg1 = fig.legend(handles=handles_main, loc="center left",
                                  bbox_to_anchor=(1.0, 0.5), frameon=True,
                                  fontsize=10, title="Legend", title_fontsize=10)
                leg1.get_frame().set_alpha(0.96)

                plt.tight_layout(rect=[0, 0, 0.85, 1])
                fig.patch.set_facecolor("whitesmoke")
                plt.savefig(outpath, dpi=120, bbox_extra_artists=(leg1,), bbox_inches='tight')
                #plt.show()
                plt.close(fig)
                gc.collect()
                print(f"  [OK] {outpath}")


# 4. Inter hirarchy

In [ ]:
# ========= Paths / Parameters =========
# [Modified] Change result save path to 'yearly_distribution'
OUT_DIR = "outputs/figures/communities/inter_hierarchy"
COMM_BASE_DIR = "outputs/gpkg/communities"  # …/{key}/{weight}_SGG_map_zone_community.gpkg
COMMUNITY_METHODS = ['Infomap', "Leiden", "Louvain"]
WEIGHTS = ["거래관계"]

# [Added] Natural Breaks classification parameters
N_CLASSES = 3  # Number of classes to divide DII into
JITTER_STRENGTH = 0.1 # Node jitter strength along X-axis


# ===== Main Loop =====
os.makedirs(OUT_DIR, exist_ok=True)

# Assume nw_dict is already loaded
for key, ed_original in nw_dict.items():
    for weight in WEIGHTS:
        gpkg_path = os.path.join(COMM_BASE_DIR, key, f"{weight}_SGG_map_zone_community.gpkg")
        if not os.path.exists(gpkg_path):
            continue

        g_comm = gpd.read_file(gpkg_path)
        g_comm["SIGUNGU_CD"] = g_comm["SIGUNGU_CD"].astype(str)

        for method in COMMUNITY_METHODS:
            print(f"--- Processing [{key} / {weight} / {method}] ---")
            
            # --- 1. Aggregate DII data across all years ---
            all_dii_records = [] # [{'year': y, 'cid': c, 'dii': d}, ...]
            
            years = sorted({int(c.split("_")[-1]) for c in g_comm.columns if c.startswith(f"{method}_canon_")})
            if not years:
                print("  [SKIP] No canonical years found.")
                continue

            # Standardize original data column names
            ed = ed_original.copy()
            rename_map = {
                '1_기준연도': 'year', '8_시군구코드_seller': 'source', '14_시군구코드_buyer': 'target',
                weight: 'weight'
            }
            ed.rename(columns={k: v for k, v in rename_map.items() if k in ed.columns}, inplace=True)
            
            required_cols = ['year', 'source', 'target', 'weight']
            if not all(c in ed.columns for c in required_cols):
                print(f"  [SKIP] Missing required columns in edge data.")
                continue

            ed_use = ed[required_cols].dropna().copy()
            ed_use['source'] = ed_use['source'].astype(str)
            ed_use['target'] = ed_use['target'].astype(str)
            ed_use['weight'] = pd.to_numeric(ed_use['weight'], errors='coerce').fillna(0)

            for year in years:
                col_canon = f"{method}_canon_{year}"
                if col_canon not in g_comm.columns:
                    continue

                s_year = g_comm[["SIGUNGU_CD", col_canon]].dropna()
                s_year[col_canon] = s_year[col_canon].astype(int)
                lab_map = s_year.set_index("SIGUNGU_CD")[col_canon].to_dict()

                ede = ed_use[ed_use["year"] == year].copy()
                if ede.empty:
                    continue
                
                ede["m_s"] = ede["source"].map(lab_map)
                ede["m_t"] = ede["target"].map(lab_map)
                ede = ede.dropna(subset=["m_s", "m_t"])
                ede["m_s"] = ede["m_s"].astype(int)
                ede["m_t"] = ede["m_t"].astype(int)
                ede = ede[ede["m_s"] != ede["m_t"]]
                
                flows = ede.groupby(["m_s", "m_t"])['weight'].sum()
                
                if not flows.empty:
                    io_sum = {}
                    for (ms, mt), val in flows.items():
                        io_sum[ms] = io_sum.get(ms, 0) + val
                        io_sum[mt] = io_sum.get(mt, 0) + val
                    
                    if not io_sum: continue
                    
                    mean_io = np.mean(list(io_sum.values()))
                    if mean_io > 0:
                        for cid, total_io in io_sum.items():
                            dii = total_io / mean_io
                            all_dii_records.append({'year': year, 'cid': cid, 'dii': dii})
            
            if not all_dii_records:
                print("  [SKIP] No DII data to plot.")
                continue

            # --- 2. Classify DII data and prepare for visualization ---
            dii_df = pd.DataFrame(all_dii_records)
            
            # Create Natural Breaks classifier
            # If fewer than 2 data points or fewer unique values than classes, adjust k
            unique_dii_count = dii_df['dii'].nunique()
            k = min(N_CLASSES, unique_dii_count)
            
            if k > 1:
                classifier = mapclassify.NaturalBreaks(dii_df['dii'], k=k)
                dii_df['dii_class'] = classifier.yb
            else: # When classification is meaningless
                dii_df['dii_class'] = 0
                classifier = None

            dii_df['dii_class'] = 3 - dii_df['dii_class'].astype(int)

            # --- 3. Scatter plot visualization ---
            outdir = os.path.join(OUT_DIR, key)
            os.makedirs(outdir, exist_ok=True)
            safe_name = lambda s: re.sub(r"[^0-9A-Za-z_-]+", "_", s)
            outpath = os.path.join(outdir, f"dii_yearly_dist_{safe_name(method)}.png")

            # Plot scatter separately by tier
            fig, ax = plt.subplots(figsize=(14, 8), dpi=100)

            marker_map = {
                1: '*',  # Tier 1
                2: 'o',  # Tier 2
                3: '^'   # Tier 3
            }

            color_map = {cid: COLOR_OF.get(cid, 'black') for cid in dii_df['cid'].unique()}

            legend_handles = []

            # Plot scatter separately by tier
            for tier, marker in marker_map.items():
                subset = dii_df[dii_df['dii_class'] == tier]
                if subset.empty:
                    continue

                # Build a list of colors for each row
                colors = subset['cid'].map(color_map)

                # Main scatter
                ax.scatter(
                    x=subset['year'],
                    y=subset['dii'],
                    c=colors,
                    s=150,
                    alpha=0.8,
                    edgecolors=colors,
                    linewidth=3,
                    zorder=10,
                    marker=marker
                )

                # --- Legend entries for each community (instead of text labels) ---
                legend_handles.append(Patch(color='none', label=''))
                for cid in subset['cid'].unique():
                    legend_handles.append(Line2D(
                        [], [], 
                        marker=marker, 
                        color=color_map[cid],
                        markersize=9, 
                        linestyle='None',
                        markeredgecolor=color_map[cid],
                        markeredgewidth=2,
                        label=f"{cid}"
                    ))

            # --- Graph style settings ---
            ax.set_title(f"Yearly DII Distribution of Communities (Natural Breaks)\n[{key} / {weight} / {method}]", fontsize=16, pad=15)
            ax.set_xlabel("Year", fontsize=12)
            ax.set_ylabel("Dominance Index (DII)", fontsize=12)
            ax.grid(True, which='major', linestyle=':', linewidth=0.5, color='gray', alpha=0.5)
            
            ax.axhline(1.0, color='red', linestyle='--', linewidth=1.2, label='Average (DII=1.0)')
            
            # Set X-axis ticks to integer years
            ax.set_xticks(years)
            plt.xticks(rotation=0)
            ax.set_ylim(bottom=0, top=4.5)
            
            # Y-axis scale (consider log scale if values are very large)
            # ax.set_yscale('log')

            # --- Create legend ---
            handles = [ax.get_legend_handles_labels()[0][0]] # Add handle for DII=1.0 line
            labels  = [ax.get_legend_handles_labels()[1][0]]
            handles.append(Patch(color='none', label='')) # Spacer
            labels.append('')

            if classifier:
                for i in range(k):
                    marker = marker_map.get(i+1, 'o')
                    lower_bound = dii_df['dii'].min() if i == 0 else classifier.bins[i-1]
                    upper_bound = classifier.bins[i]
                    tier_label = f"Tier {i+1} ({lower_bound:.2f} ~ {upper_bound:.2f})"

                    # Tier 범위 라벨 (섹션 헤더 역할)
                    handles.append(Patch(color='none', label=tier_label))
                    labels.append(tier_label)

                    # 해당 Tier에 속한 community들 추가
                    subset = dii_df[dii_df['dii_class'] == (i+1)]
                    for cid in subset['cid'].unique():
                        handles.append(Line2D([], [], 
                                            marker=marker,
                                            color=color_map[cid],
                                            markersize=9,
                                            linestyle='None',
                                            markeredgecolor=color_map[cid],
                                            markeredgewidth=2,
                                            label=f"M{cid}"))
                        labels.append(f"M{cid}")

            ax.legend(handles=handles, labels=labels,
                    loc='center left', bbox_to_anchor=(1.02, 0.5),
                    title="Legend", fontsize=10)


            plt.tight_layout(rect=[0, 0, 0.88, 1]) # Adjust layout so legend isn’t cut off
            ax.patch.set_facecolor("whitesmoke")

            plt.savefig(outpath, dpi=100, bbox_inches='tight')
            #plt.show()
            plt.close(fig)
            gc.collect()
            print(f"  [OK] Saved DII distribution plot to {outpath}")
